In [7]:
# Check what database/catalog we are currently using
print("current_catalog:", spark.sql("SELECT current_catalog()").collect()[0][0])
print("current_database:", spark.sql("SELECT current_database()").collect()[0][0])




StatementMeta(, 144748da-eddf-4d95-b372-c29822103f9b, 10, Finished, Available, Finished)

current_catalog: spark_catalog
current_database: chimcobldhq2ajbp41rmusjbedo62or54lo6usjkcpnmoqbfbto74qbdc5p7inr3c5p6anrcc5lmaq3felpma9b4c9ng


In [8]:
spark.sql("CREATE SCHEMA IF NOT EXISTS bronze")


StatementMeta(, 144748da-eddf-4d95-b372-c29822103f9b, 11, Finished, Available, Finished)

DataFrame[]

In [1]:
# Check if schemea is created
spark.sql("SHOW DATABASES").show(50, truncate=False) 


StatementMeta(, 8648cb5e-af81-45c4-8cf1-dca36cf66dc7, 3, Finished, Available, Finished)

+----------------------------------------------------+
|namespace                                           |
+----------------------------------------------------+
|My workspace.portfolio_primary_care_lakehouse.bronze|
|My workspace.portfolio_primary_care_lakehouse.dbo   |
+----------------------------------------------------+



In [4]:
from pyspark.sql.functions import col

# Read raw CSV
df_raw = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "false")
    .csv("Files/bronze/appointments/appointments_raw.csv")
)

# Minimal, mechanical column name standardisation for Delta compatibility
df_raw_standardised = (
    df_raw
    .withColumnRenamed("EMIS Number", "emis_number")
    .withColumnRenamed("Usual GP (Extended Services)'s Organisation Name", "usual_gp_org_name")
    .withColumnRenamed("Usual GP (Extended Services)'s National Practice Code", "practice_code")
    .withColumnRenamed("Appointment Date", "appointment_date")
    .withColumnRenamed("Appointment Time", "appointment_time")
    .withColumnRenamed("Slot Type", "slot_type")
    .withColumnRenamed("Current Slot Status", "current_slot_status")
    .withColumnRenamed("Location's Name", "location_name")
    .withColumnRenamed("Session Holder", "session_holder")
)

# Write to Bronze Delta table
df_raw_standardised.write.mode("overwrite").saveAsTable("bronze.appointments_raw")



StatementMeta(, 8648cb5e-af81-45c4-8cf1-dca36cf66dc7, 6, Finished, Available, Finished)

In [6]:
## Checking that all columns are strings
df_raw_standardised.printSchema()

StatementMeta(, 8648cb5e-af81-45c4-8cf1-dca36cf66dc7, 8, Finished, Available, Finished)

root
 |-- emis_number: string (nullable = true)
 |-- usual_gp_org_name: string (nullable = true)
 |-- practice_code: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- appointment_date: string (nullable = true)
 |-- appointment_time: string (nullable = true)
 |-- slot_type: string (nullable = true)
 |-- current_slot_status: string (nullable = true)
 |-- location_name: string (nullable = true)
 |-- session_holder: string (nullable = true)



In [10]:
## Comparing count of rows in csv to those in bronze table
csv_count = (
    spark.read
    .option("header", "true")
    .csv("Files/bronze/appointments/appointments_raw.csv")
    .count()
)

csv_count

StatementMeta(, 8648cb5e-af81-45c4-8cf1-dca36cf66dc7, 12, Finished, Available, Finished)

10

In [11]:
## Count rows in Bronze table
table_count = spark.table("bronze.appointments_raw").count()
table_count

StatementMeta(, 8648cb5e-af81-45c4-8cf1-dca36cf66dc7, 13, Finished, Available, Finished)

10